In [1]:
%load_ext tensorboard

In [2]:
#standard packages
import gym
import numpy as np
import cv2
import torch as th
from torch import nn
import os
from tqdm import tqdm

# mario packages
import gym_super_mario_bros
from gym_super_mario_bros import SuperMarioBrosEnv
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import *

# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation

# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

# Import algo
from stable_baselines3 import A2C, PPO

# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import VecVideoRecorder, SubprocVecEnv, DummyVecEnv

In [3]:
tensorboard_logdir = os.path.abspath(".")
# reward_log_path = os.path.join(tensorboard_logdir, 'reward_log.csv')

In [53]:
%tensorboard --logdir $tensorboard_logdir

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

# Regular Environment

In [6]:
tensorboard_logdir = os.path.abspath("./mario/reg_model")
reward_log_path = os.path.join(tensorboard_logdir, 'reward_log.csv')

In [7]:
%tensorboard --logdir $tensorboard_logdir --port=6008

Reusing TensorBoard on port 6008 (pid 20216), started 19:05:05 ago. (Use '!kill 20216' to kill it.)

# Check reward

In [4]:
import pandas as pd

In [5]:
reward_ccmodel = pd.read_csv("mario/cc_model/reward_log.csv")
reward_ccmodel = reward_ccmodel.loc[reward_ccmodel.timesteps!="timesteps", :].reset_index(drop=True)
reward_ccmodel["timesteps"] = reward_ccmodel["timesteps"].astype("float")
reward_ccmodel["best_reward"] = reward_ccmodel["best_reward"].astype("float")
reward_ccmodel = reward_ccmodel.sort_values(by="best_reward", ascending=False)
print(reward_ccmodel.head(10))
print(reward_ccmodel.sort_values(by="timesteps", ascending=False).head(5))
# reward_ccmodel.loc[reward_ccmodel.timesteps=="timesteps", :]

      timesteps     reward  best_reward
2328  2328000.0   1857.85        3080.0
2120  2120000.0   2141.05        3079.0
1749  1749000.0    2303.9        3077.0
2370  2370000.0    1960.9        3077.0
1418  1418000.0    2113.7        3077.0
2489  2489000.0    2031.2        3077.0
2255  2255000.0   1991.75        3076.0
1375  1375000.0    1837.3        3076.0
1934  1934000.0   2331.95        3075.0
2147  2147000.0   2091.45        3075.0
      timesteps     reward  best_reward
2621  2621000.0    2220.5        3069.0
2620  2620000.0   1775.05        3061.0
2619  2619000.0   2388.55        3062.0
2618  2618000.0    1835.3        3067.0
2617  2617000.0   1901.75        3057.0


In [15]:
reward_regmodel = pd.read_csv("mario/reg_model/reward_log.csv").sort_values(by="best_reward", ascending=False)
print(reward_regmodel.head(10))
print(reward_regmodel.sort_values(by="timesteps", ascending=False).head(5))

      timesteps   reward  best_reward
2272    2273000  2289.75       3068.0
2447    2448000  2649.50       3068.0
2345    2346000  2287.05       3068.0
2346    2347000  2334.40       3068.0
2351    2352000  2586.05       3068.0
2443    2444000  2557.75       3068.0
2352    2353000  2543.20       3068.0
2425    2426000  2628.05       3068.0
2519    2520000  2150.35       3068.0
2424    2425000  2517.30       3068.0
      timesteps   reward  best_reward
2523    2524000  2163.10       3062.0
2522    2523000  2032.20       3062.0
2521    2522000  2332.50       3063.0
2520    2521000  2584.80       3062.0
2519    2520000  2150.35       3068.0


In [7]:
reward_ccmodel_a2c = pd.read_csv("../actorcritic/mario/cc_model/reward_log.csv").sort_values(by="best_reward", ascending=False)
print(reward_ccmodel_a2c.head(10))
print(reward_ccmodel_a2c.sort_values(by="timesteps", ascending=False).head(5))

     timesteps  reward  best_reward
329     330000  779.45       2971.0
70       71000  629.80       2661.0
351     352000  827.55       2445.0
346     347000  710.35       2293.0
322     323000  585.45       2288.0
312     313000  751.90       2264.0
324     325000  619.85       1861.0
119     120000  643.50       1827.0
338     339000  824.90       1820.0
344     345000  794.70       1814.0
     timesteps  reward  best_reward
388     389000  596.30       1314.0
387     388000  492.15        938.0
386     387000  577.30       1053.0
385     386000  516.30        966.0
384     385000  575.40       1263.0


In [16]:
reward_regmodel_a2c_df1 = pd.read_csv("../actorcritic/mario/reg_model/reward.csv").sort_values(by="best_reward", ascending=False)
reward_regmodel_a2c_df2 = pd.read_csv("../actorcritic/mario/reg_model/reward_log.csv")
reward_regmodel_a2c_df2.columns = reward_regmodel_a2c_df1.columns
reward_regmodel_a2c_df2 = reward_regmodel_a2c_df2.sort_values(by="best_reward", ascending=False)
reward_regmodel_a2c = pd.concat([reward_regmodel_a2c_df1, reward_regmodel_a2c_df2], axis=0)
print(reward_regmodel_a2c.head(10))
print(reward_regmodel_a2c.sort_values(by="timesteps", ascending=False).head(5))

     timesteps  reward  best_reward
218     219000  751.95       2245.0
191     192000  684.15       2030.0
229     230000  660.80       1864.0
217     218000  639.50       1803.0
207     208000  660.15       1615.0
215     216000  650.70       1607.0
278     279000  717.05       1599.0
39       40000  558.15       1592.0
216     217000  658.45       1565.0
205     206000  615.25       1492.0
    timesteps  reward  best_reward
69     357000  588.75       1195.0
68     356000  578.35       1035.0
67     355000  430.50        915.0
66     354000  482.20        733.0
65     353000  584.70       1212.0


# Check results

In [37]:
#standard packages
import gym
import numpy as np
import cv2
import torch as th
from torch import nn
import os

# mario packages
import gym_super_mario_bros
from gym_super_mario_bros import SuperMarioBrosEnv
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import *
from gym.wrappers import RecordVideo

# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation

# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

# Import algo
from stable_baselines3 import A2C, PPO

# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import VecVideoRecorder, SubprocVecEnv, DummyVecEnv

class CoinCollectorSuperMarioBrosEnv(SuperMarioBrosEnv):
    #score btn 2 time frames can maybe go upto 8000 so we can just divide by 100 (reference https://www.mariowiki.com/Point)
    reward_range = (-15, 100)

    def __init__(self, rom_mode='vanilla', lost_levels=False, target=None):
        super().__init__(rom_mode=rom_mode, lost_levels=lost_levels, target=target)

        # variable to keep track of score deltas
        self._score_last = 0

    @property
    def _score_reward(self):
        _reward = self._score - self._score_last
        self._score_last = self._score
        return _reward/100

    # This should override the parent function
    def _get_reward(self):
        return self._x_reward + self._score_reward + self._time_penalty + self._death_penalty

'''
The code below registers this new environment in gym for us to reference later. Code borrowed from _registration.py of gym_super_mario_bros
'''
def _register_coin_collector_mario_stage_env(id, **kwargs):
    """
    Register a Super Mario Bros. (1/2) stage environment with OpenAI Gym.

    Args:
        id (str): id for the env to register
        kwargs (dict): keyword arguments for the SuperMarioBrosEnv initializer

    Returns:
        None

    """
    # register the environment
    gym.envs.registration.register(
        id=id,
        # entry_point='.:CoinCollectorSuperMarioBrosEnv',
        entry_point=CoinCollectorSuperMarioBrosEnv,
        max_episode_steps=9999999,
        reward_threshold=9999999,
        kwargs=kwargs,
        nondeterministic=True,
    )

def _register_all_coin_collector_envs():
    # a template for making individual stage environments
    _ID_TEMPLATE = 'CoinCollectorSuperMarioBrosEnv-{}-{}-v{}'
    # A list of ROM modes for each level environment
    _ROM_MODES = [
        'vanilla',
        'downsample',
        'pixel',
        'rectangle'
    ]

    # iterate over all the rom modes, worlds (1-8), and stages (1-4)
    for version, rom_mode in enumerate(_ROM_MODES):
        for world in range(1, 9):
            for stage in range(1, 5):
                # create the target
                target = (world, stage)
                # setup the frame-skipping environment
                env_id = _ID_TEMPLATE.format(world, stage, version)
                print(f"Registering Coin Collector {env_id} in gym for use later on.")
                _register_coin_collector_mario_stage_env(env_id, rom_mode=rom_mode, target=target)
                print(f"Successfully registered coin collector env {env_id}!")

def create_gym_env_from_level(world, stage, version, use_coin_collector_env):
    level_suffix = f"{world}-{stage}-v{version}"
    if not use_coin_collector_env:
        level = f"SuperMarioBros-{level_suffix}"
        env = gym_super_mario_bros.make(level)
    else:
        env_set = set(gym.envs.registration.registry.env_specs.copy().keys())
        level = f"CoinCollectorSuperMarioBrosEnv-{level_suffix}"
        if level not in env_set:
            # register all these custom environments for the first time
            _register_all_coin_collector_envs()

        assert level in set(
            gym.envs.registration.registry.env_specs.copy().keys()
        ), f"Looks like {level} was not registered correctly!"
        env = gym.make(level)

    return env

class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

class ResizeEnv(gym.ObservationWrapper):
    def __init__(self, env, size):
        gym.ObservationWrapper.__init__(self, env)
        (oldh, oldw, oldc) = env.observation_space.shape
        newshape = (size, size, oldc)
        self.observation_space = gym.spaces.Box(low=0, high=255,
            shape=newshape, dtype=np.uint8)

    def observation(self, frame):
        height, width, _ = self.observation_space.shape
        frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
        if frame.ndim == 2:
            frame = frame[:,:,None]
        return frame

def create_mario_env(world, stage, version, use_coin_collector_env):
    env = create_gym_env_from_level(world, stage, version, use_coin_collector_env)
    env = JoypadSpace(env, COMPLEX_MOVEMENT)
    env = SkipFrame(env, skip=4)
    env = GrayScaleObservation(env, keep_dim=True)
    env = ResizeEnv(env, size=84)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, 4, channels_order='last')
    return env

class MarioNet(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim):
        super(MarioNet, self).__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(th.as_tensor(observation_space.sample()[None]).float()).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))


In [25]:
# <world> is a number in {1, 2, 3, 4, 5, 6, 7, 8} indicating the world
world = 1
# <stage> is a number in {1, 2, 3, 4} indicating the stage within a world
stage = 1
version = 3
use_coin_collector_env = True

env = create_mario_env(world, stage, version, use_coin_collector_env)

env.reset()
state, reward, done, info = env.step([0])
print('state:', state.shape) #Color scale, height, width, num of stacks


env = create_mario_env(world, stage, version, use_coin_collector_env)
plays = 4

state: (1, 84, 84, 4)


In [1]:
# best_epoch = 1418000 #change as per the values inferred from the graph
# best_model_path = 'mario/cc_model/best_model_{}.zip'.format(best_epoch)

# # Load the best model
# model = PPO.load(best_model_path)
# record_env = RecordVideo(env, f"mario_plays/cc_model/{best_epoch}", name_prefix="mario_ppo")

# for ep in range(plays):
#     state = record_env.reset()
#     done = False
#     while not done:
#         action, _ = model.predict(state)
#         state, reward, done, info = record_env.step(action)
#         record_env.render()
# print(f"Episode {ep} done")

In [23]:
info[0]["flag_get"]

False

In [81]:
# <world> is a number in {1, 2, 3, 4, 5, 6, 7, 8} indicating the world
world = 1
# <stage> is a number in {1, 2, 3, 4} indicating the stage within a world
stage = 1
version = 3
use_coin_collector_env = False

env = create_mario_env(world, stage, version, use_coin_collector_env)

env.reset()
state, reward, done, info = env.step([0])
print('state:', state.shape) #Color scale, height, width, num of stacks


env = create_mario_env(world, stage, version, use_coin_collector_env)
plays = 4

state: (1, 84, 84, 4)


In [85]:
best_epoch = 848000 #change as per the values inferred from the graph
best_model_path = 'mario/reg_model/best_model_{}.zip'.format(best_epoch)

# Load the best model
model = PPO.load(best_model_path)
record_env = RecordVideo(env, f"mario_plays/reg_model/{best_epoch}", name_prefix="mario_ppo")

for ep in range(plays):
    state = record_env.reset()
    done = False
    while not done:
        action, _ = model.predict(state)
        state, reward, done, info = record_env.step(action)
        record_env.render()
        print(f"Episode {ep} done")

Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 done
Episode 0 

## Best model record plays

In [40]:
best_cc_model_timestep = 2328000
best_reg_model_timestep = 2273000

best_cc_model = 'mario/cc_model/best_model_{}.zip'.format(best_cc_model_timestep)
best_cc_model = PPO.load(best_cc_model)

best_reg_model = 'mario/reg_model/best_model_{}.zip'.format(best_reg_model_timestep)
best_reg_model = PPO.load(best_reg_model)

In [58]:
def pass_rate(model=None, plays=1000, world=1, stage=1, version=3, use_coin_collector_env=True):
    get_count = 0
    coin_count = 0
    coin_collection = []

    env = env_setup(world=world, stage=stage, version=version, use_coin_collector_env=use_coin_collector_env)

    for _ in tqdm(range(plays)):
        state = env.reset()
        done = False
        while not done:
            action, _ = model.predict(state)
            state, reward, done, info = env.step(action)
        if info[0]['flag_get'] == True:
            get_count += 1
        coin_count_current_play = info[0]['coins']
        coin_collection.append(coin_count_current_play)
        coin_count += info[0]['coins']
    # print(f'flag get count: {get_count}')
    # print(f'total coin count: {coin_count}')

    return get_count, coin_count, coin_collection

def env_setup(world=1, stage=1, version=3, use_coin_collector_env=True):

    env = create_mario_env(world=world, stage=stage, version=version, use_coin_collector_env=use_coin_collector_env)
    env.reset()
    state, reward, done, info = env.step([0])
    print('state:', state.shape) #Color scale, height, width, num of stacks

    env = create_mario_env(world=world, stage=stage, version=version, use_coin_collector_env=use_coin_collector_env)
    return env

In [42]:
# cc model trial 1
pass_count_cc, coin_count_cc, coin_collected_t1_cc = pass_rate(best_cc_model, use_coin_collector_env=True)
print(pass_count_cc, coin_count_cc)

371 996


In [59]:
# cc model trial 2
pass_count_cc, coin_count_cc, coin_collected_t2_cc = pass_rate(best_cc_model, use_coin_collector_env=True)
print(pass_count_cc, coin_count_cc)

state: (1, 84, 84, 4)


100%|██████████| 1000/1000 [44:44<00:00,  2.68s/it] 

356 1036


In [60]:
# cc model trial 3
pass_count_cc, coin_count_cc, coin_collected_t3_cc = pass_rate(best_cc_model, use_coin_collector_env=True)
print(pass_count_cc, coin_count_cc)

state: (1, 84, 84, 4)


100%|██████████| 1000/1000 [44:23<00:00,  2.66s/it] 

343 1005


In [47]:
# reg model trial 1
pass_count_reg, coin_count_reg = pass_rate(best_reg_model, use_coin_collector_env=False)
print(pass_count_reg, coin_count_reg)

374 700


In [61]:
# reg model trial 2
pass_count_reg, coin_count_reg, coin_collected_t2_reg = pass_rate(best_reg_model, use_coin_collector_env=False)
print(pass_count_reg, coin_count_reg)

state: (1, 84, 84, 4)


100%|██████████| 1000/1000 [48:48<00:00,  2.93s/it] 

382 667


In [62]:
# reg model trial 3
pass_count_reg, coin_count_reg, coin_collected_t3_cc = pass_rate(best_reg_model, use_coin_collector_env=False)
print(pass_count_reg, coin_count_reg)

state: (1, 84, 84, 4)


100%|██████████| 1000/1000 [47:59<00:00,  2.88s/it] 

369 622
